In [ ]:
#| default_exp losses

# Losses

> Loss functions for model evaluation.
> 

> The most important train signal is the forecast error, which is the difference between the observed value $y_{\tau}$ and the prediction $\hat{y}_{\tau}$, at time $y_{\tau}$:
> 
> $$ e_{\tau} = y_{\tau}-\hat{y}_{\tau} \qquad \qquad \tau \in \{t+1,\dots,t+H \} $$
> 
> The train loss summarizes the forecast errors in different evaluation metrics.

In [ ]:
#| export
from functools import wraps
from typing import Callable, List, Optional, Union

import numpy as np
import pandas as pd

from utilsforecast.compat import DataFrame

In [ ]:
#| hide
import warnings

from nbdev import show_doc

In [ ]:
#| hide
warnings.filterwarnings('ignore', message='Unknown section References')

In [ ]:
#| exporti
def _divide_no_nan(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Auxiliary funtion to handle divide by 0"""
    out_dtype = np.result_type(np.float32, a.dtype, b.dtype)
    return np.divide(a, b, out=np.zeros(a.shape, dtype=out_dtype), where=b != 0)

In [ ]:
#| exporti
def _metric_protections(
    y: np.ndarray, y_hat: np.ndarray, weights: Optional[np.ndarray] = None
) -> None:
    if weights is None:
        return
    if np.sum(weights) <= 0:
        raise ValueError('Sum of weights must be positive')
    if y.shape != y_hat.shape:
        raise ValueError(f'Wrong y_hat dimension. y_hat shape={y_hat.shape}, y shape={y.shape}')
    if weights.shape != y.shape:
        raise ValueError(f'Wrong weight dimension. weights shape={weights.shape}, y shape={y.shape}')

In [ ]:
import polars as pl

from utilsforecast.data import generate_series

In [ ]:
series = generate_series(10, static_as_categorical=False)
rng = np.random.RandomState(0)
models = ['model1', 'model2']
for model in models:
    series[model] = series['y'] * rng.rand(series.shape[0])
series_pl = pl.from_pandas(series)

# <span style="color:DarkOrange">1. Scale-dependent Errors </span>

## Mean Absolute Error (MAE)
> $$ \mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} |y_{\tau} - \hat{y}_{\tau}| $$

![](imgs/losses/mae_loss.png)

In [ ]:
#| exporti
def _base_docstring(*args, **kwargs) -> Callable:
    base_docstring = """

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    model_cols : list of str
        Columns that identify the models predictions.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with the {name} for each id.
    """
    def docstring_decorator(f: Callable):
        f.__doc__ = f.__doc__ + base_docstring.format(name=f.__name__.upper())
        return f

    return docstring_decorator(*args, **kwargs)

In [ ]:
#| export
@_base_docstring
def mae(
    df: DataFrame,
    model_cols: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DataFrame:
    """Mean Absolute Error (MAE)

    MAE measures the relative prediction
    accuracy of a forecasting method by calculating the
    deviation of the prediction and the true
    value at a given time and averages these devations
    over the length of the series."""
    if isinstance(df, pd.DataFrame):
        res = (df[model_cols].sub(df[target_col], axis=0)).abs().groupby(df[id_col], observed=True).mean()
        res.index.name = id_col
        res = res.reset_index()
    else:
        exprs = [(pl.col(target_col) - pl.col(model_col)).abs().mean().alias(model_col) for model_col in model_cols]
        res = df.group_by(id_col).agg(exprs)
    return res

In [ ]:
show_doc(mae)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L36){target="_blank" style="float:right; font-size:smaller"}

### mae

>      mae
>           (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFra
>           me], model_cols:List[str], id_col:str='unique_id',
>           target_col:str='y')

Mean Absolute Error (MAE)

MAE measures the relative prediction
accuracy of a forecasting method by calculating the
deviation of the prediction and the true
value at a given time and averages these devations
over the length of the series.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| model_cols | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **Union** |  | **dataframe with the MAE for each id.** |

In [ ]:
mae(series, models)

,unique_id,model1,model2
0,0,1.621445,1.661438
1,1,1.550644,1.425915
2,2,1.608086,1.642708
3,3,1.712587,1.600349
4,4,1.620040,1.606156
5,5,1.529447,1.765802
6,6,1.518084,1.744659
7,7,1.673386,1.686614
8,8,1.625683,1.435187
9,9,1.556535,1.644697


In [ ]:
mae(series_pl, models).sort('unique_id')

unique_id,model1,model2
i64,f64,f64
0,1.621445,1.661438
1,1.550644,1.425915
2,1.608086,1.642708
3,1.712587,1.600349
4,1.62004,1.606156
5,1.529447,1.765802
6,1.518084,1.744659
7,1.673386,1.686614
8,1.625683,1.435187


## Mean Squared Error
> $$ \mathrm{MSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} (y_{\tau} - \hat{y}_{\tau})^{2} $$

![](imgs/losses/mse_loss.png)

In [ ]:
#| export
@_base_docstring
def mse(
    df: DataFrame,
    model_cols: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DataFrame:
    """Mean Squared Error (MSE)
    
    MSE measures the relative prediction
    accuracy of a forecasting method by calculating the 
    squared deviation of the prediction and the true
    value at a given time, and averages these devations
    over the length of the series."""    
    if isinstance(df, pd.DataFrame):
        res = (df[model_cols].sub(df[target_col], axis=0)).pow(2).groupby(df[id_col], observed=True).mean()
        res.index.name = id_col
        res = res.reset_index()
    else:
        exprs = [(pl.col(target_col) - pl.col(model_col)).pow(2).mean().alias(model_col) for model_col in model_cols]        
        res = df.group_by(id_col).agg(*exprs)
    return res

In [ ]:
show_doc(mse)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L80){target="_blank" style="float:right; font-size:smaller"}

### mse

>      mse
>           (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFra
>           me], model_cols:List[str], id_col:str='unique_id',
>           target_col:str='y')

Mean Squared Error (MSE)

MSE measures the relative prediction
accuracy of a forecasting method by calculating the 
squared deviation of the prediction and the true
value at a given time, and averages these devations
over the length of the series.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| model_cols | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **Union** |  | **dataframe with the MSE for each id.** |

In [ ]:
mse(series, models)

,unique_id,model1,model2
0,0,4.924752,5.223291
1,1,4.491915,3.890422
2,2,4.714431,4.824025
3,3,5.459162,4.646057
4,4,4.832107,4.704585
5,5,4.474070,5.531093
6,6,4.520140,5.293929
7,7,5.200671,5.201971
8,8,4.087936,3.828350
9,9,4.503553,4.761206


In [ ]:
mse(series_pl, models).sort('unique_id')

unique_id,model1,model2
i64,f64,f64
0,4.924752,5.223291
1,4.491915,3.890422
2,4.714431,4.824025
3,5.459162,4.646057
4,4.832107,4.704585
5,4.47407,5.531093
6,4.52014,5.293929
7,5.200671,5.201971
8,4.087936,3.82835


## Root Mean Squared Error
> $$ \mathrm{RMSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \sqrt{\frac{1}{H} \sum^{t+H}_{\tau=t+1} (y_{\tau} - \hat{y}_{\tau})^{2}} $$

![](imgs/losses/rmse_loss.png)

In [ ]:
#| export
@_base_docstring
def rmse(
    df: DataFrame,
    model_cols: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> Union[float, np.ndarray]:
    """Root Mean Squared Error (RMSE)
    
    RMSE measures the relative prediction
    accuracy of a forecasting method by calculating the squared deviation
    of the prediction and the observed value at a given time and
    averages these devations over the length of the series.
    Finally the RMSE will be in the same scale
    as the original time series so its comparison with other
    series is possible only if they share a common scale. 
    RMSE has a direct connection to the L2 norm."""    
    res = mse(df, model_cols, id_col, target_col)
    if isinstance(res, pd.DataFrame):
        res[model_cols] = res[model_cols].pow(0.5)
    else:
        import polars as pl

        res = res.with_columns(*[pl.col(c).pow(0.5) for c in model_cols])
    return res

In [ ]:
show_doc(rmse)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L126){target="_blank" style="float:right; font-size:smaller"}

### rmse

>      rmse
>            (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFr
>            ame], model_cols:List[str], id_col:str='unique_id',
>            target_col:str='y')

Root Mean Squared Error (RMSE)

RMSE measures the relative prediction
accuracy of a forecasting method by calculating the squared deviation
of the prediction and the observed value at a given time and
averages these devations over the length of the series.
Finally the RMSE will be in the same scale
as the original time series so its comparison with other
series is possible only if they share a common scale. 
RMSE has a direct connection to the L2 norm.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| model_cols | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **Union** |  | **dataframe with the RMSE for each id.** |

In [ ]:
rmse(series, models)

,unique_id,model1,model2
0,0,2.219178,2.285452
1,1,2.119414,1.972415
2,2,2.171274,2.196366
3,3,2.336485,2.155471
4,4,2.198205,2.169005
5,5,2.115200,2.351828
6,6,2.126062,2.300854
7,7,2.280498,2.280783
8,8,2.021865,1.956617
9,9,2.122158,2.182019


In [ ]:
rmse(series_pl, models).sort('unique_id')

unique_id,model1,model2
i64,f64,f64
0,2.219178,2.285452
1,2.119414,1.972415
2,2.171274,2.196366
3,2.336485,2.155471
4,2.198205,2.169005
5,2.1152,2.351828
6,2.126062,2.300854
7,2.280498,2.280783
8,2.021865,1.956617


# <span style="color:DarkOrange">2. Percentage Errors </span>

## Mean Absolute Percentage Error
> $$ \mathrm{MAPE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{|y_{\tau}|} $$

![](imgs/losses/mape_loss.png)

In [ ]:
#| export
@_base_docstring
def mape(
    df: DataFrame,
    model_cols: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> Union[float, np.ndarray]:
    """Mean Absolute Percentage Error (MAPE)
    
    MAPE measures the relative prediction
    accuracy of a forecasting method by calculating the percentual deviation
    of the prediction and the observed value at a given time and
    averages these devations over the length of the series.
    The closer to zero an observed value is, the higher penalty MAPE loss
    assigns to the corresponding error."""
    if isinstance(df, pd.DataFrame):
        res = df[model_cols].sub(df[target_col], axis=0).abs().div(df[target_col].abs(), axis=0).groupby(df[id_col], observed=True).mean()
        res.index.name = id_col
        res = res.reset_index()
    else:
        exprs = [(pl.col(target_col).sub(pl.col(model_col)).abs() / pl.col(target_col).abs()).mean().alias(model_col) for model_col in model_cols]        
        res = df.group_by(id_col).agg(*exprs)
    return res

In [ ]:
show_doc(mape)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L165){target="_blank" style="float:right; font-size:smaller"}

### mape

>      mape
>            (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFr
>            ame], model_cols:List[str], id_col:str='unique_id',
>            target_col:str='y')

Mean Absolute Percentage Error (MAPE)

MAPE measures the relative prediction
accuracy of a forecasting method by calculating the percentual deviation
of the prediction and the observed value at a given time and
averages these devations over the length of the series.
The closer to zero an observed value is, the higher penalty MAPE loss
assigns to the corresponding error.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| model_cols | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **Union** |  | **dataframe with the MAPE for each id.** |

In [ ]:
mape(series, models)

,unique_id,model1,model2
0,0,0.500683,0.496882
1,1,0.508226,0.435761
2,2,0.506888,0.517446
3,3,0.508849,0.499527
4,4,0.505346,0.497153
5,5,0.474170,0.522293
6,6,0.468427,0.544866
7,7,0.510691,0.514513
8,8,0.499559,0.463213
9,9,0.482473,0.512871


In [ ]:
mape(series_pl, models).sort('unique_id')

unique_id,model1,model2
i64,f64,f64
0,0.500683,0.496882
1,0.508226,0.435761
2,0.506888,0.517446
3,0.508849,0.499527
4,0.505346,0.497153
5,0.47417,0.522293
6,0.468427,0.544866
7,0.510691,0.514513
8,0.499559,0.463213


## Symmetric Mean Absolute Percentage Error
> $$ \mathrm{SMAPE}_{2}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{|y_{\tau}|+|\hat{y}_{\tau}|} $$

In [ ]:
#| export
@_base_docstring
def smape(
    df: DataFrame,
    model_cols: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> Union[float, np.ndarray]:
    """Symmetric Mean Absolute Percentage Error (SMAPE)

    SMAPE measures the relative prediction
    accuracy of a forecasting method by calculating the relative deviation
    of the prediction and the observed value scaled by the sum of the
    absolute values for the prediction and observed value at a
    given time, then averages these devations over the length
    of the series. This allows the SMAPE to have bounds between
    0% and 200% which is desireble compared to normal MAPE that
    may be undetermined when the target is zero."""
    if isinstance(df, pd.DataFrame):
        delta_y = df[model_cols].sub(df[target_col], axis=0).abs()
        scale = df[model_cols].abs().add(df[target_col].abs(), axis=0)
        raw = delta_y.div(scale).fillna(0)
        res = raw.groupby(df[id_col], observed=True).mean()
        res.index.name = id_col
        res = res.reset_index()
    else:
        exprs = [(pl.col(model_col).sub(pl.col(target_col)).abs().truediv(pl.col(model_col).abs().add(pl.col(target_col).abs()))).fill_nan(0).alias(model_col) for model_col in model_cols]
        res = df.select([id_col, *exprs]).group_by(id_col).mean()
    return res

In [ ]:
show_doc(smape)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L209){target="_blank" style="float:right; font-size:smaller"}

### smape

>      smape
>             (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataF
>             rame], model_cols:List[str], id_col:str='unique_id',
>             target_col:str='y')

Symmetric Mean Absolute Percentage Error (SMAPE)

SMAPE measures the relative prediction
accuracy of a forecasting method by calculating the relative deviation
of the prediction and the observed value scaled by the sum of the
absolute values for the prediction and observed value at a
given time, then averages these devations over the length
of the series. This allows the SMAPE to have bounds between
0% and 200% which is desireble compared to normal MAPE that
may be undetermined when the target is zero.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| model_cols | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **Union** |  | **dataframe with the SMAPE for each id.** |

In [ ]:
smape(series, models)

,unique_id,model1,model2
0,0,0.385889,0.383957
1,1,0.400748,0.320797
2,2,0.394142,0.399449
3,3,0.395298,0.387304
4,4,0.392635,0.383696
5,5,0.365222,0.407219
6,6,0.360065,0.426810
7,7,0.399843,0.398527
8,8,0.372206,0.344200
9,9,0.367905,0.394384


In [ ]:
smape(series_pl, models).sort('unique_id')

unique_id,model1,model2
i64,f64,f64
0,0.385889,0.383957
1,0.400748,0.320797
2,0.394142,0.399449
3,0.395298,0.387304
4,0.392635,0.383696
5,0.365222,0.407219
6,0.360065,0.42681
7,0.399843,0.398527
8,0.372206,0.3442


# <span style="color:DarkOrange">3. Scale-independent Errors </span>

## Mean Absolute Scaled Error
> $$ \mathrm{MASE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau}) = 
        \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau})} $$

![](imgs/losses/mase_loss.png)

In [ ]:
#| export
@_base_docstring
def mase(
    df: DataFrame,
    model_cols: List[str],
    seasonality: int,
    train_df: DataFrame,
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DataFrame:
    """Mean Absolute Scaled Error (MASE)
    
    MASE measures the relative prediction
    accuracy of a forecasting method by comparinng the mean absolute errors
    of the prediction and the observed value against the mean
    absolute errors of the seasonal naive model.
    The MASE partially composed the Overall Weighted Average (OWA), 
    used in the M4 Competition."""    
    if isinstance(df, pd.DataFrame):
        res = df[model_cols].sub(df[target_col], axis=0).abs().groupby(df[id_col], observed=True).mean()
        # assume train_df is sorted
        lagged = train_df.groupby(id_col, observed=True)[target_col].shift(seasonality)
        scale = (train_df[target_col] - lagged).abs().groupby(train_df[id_col], observed=True).mean()
        res = res.div(scale, axis=0)
        res.index.name = id_col
        res = res.reset_index()
    else:
        exprs = [(pl.col(target_col).sub(pl.col(model_col)).abs()).mean().alias(model_col) for model_col in model_cols]
        res = df.group_by(id_col).agg(*exprs) 
        # assume train_df is sorted
        expr = (pl.col(target_col).sub(pl.col(target_col).shift(seasonality)).abs()).mean().alias('scale')
        scale = train_df.group_by(id_col).agg(expr)
        res = res.join(scale, on=id_col, how='left').select([id_col, *[(pl.col(model_col) / pl.col('scale')).alias(model_col) for model_col in model_cols]])
    return res

In [ ]:
show_doc(mase)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L260){target="_blank" style="float:right; font-size:smaller"}

### mase

>      mase
>            (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFr
>            ame], model_cols:List[str], seasonality:int, train_df:Union[pandas.
>            core.frame.DataFrame,polars.dataframe.frame.DataFrame],
>            id_col:str='unique_id', target_col:str='y')

Mean Absolute Scaled Error (MASE)

MASE measures the relative prediction
accuracy of a forecasting method by comparinng the mean absolute errors
of the prediction and the observed value against the mean
absolute errors of the seasonal naive model.
The MASE partially composed the Overall Weighted Average (OWA), 
used in the M4 Competition.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| model_cols | List |  | Columns that identify the models predictions. |
| seasonality | int |  |  |
| train_df | Union |  |  |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **Union** |  | **dataframe with the MASE for each id.** |

In [ ]:
mase(series, models, 7, series)

,unique_id,model1,model2
0,0,10.108787,10.358125
1,1,8.942334,8.223041
2,2,9.382295,9.584296
3,3,10.709936,10.008035
4,4,9.304687,9.224945
5,5,8.907966,10.284571
6,6,8.513493,9.784138
7,7,10.174744,10.255175
8,8,11.217429,9.902984
9,9,8.979968,9.488590


In [ ]:
mase(series_pl, models, 7, series_pl).sort('unique_id')

unique_id,model1,model2
i64,f64,f64
0,10.108787,10.358125
1,8.942334,8.223041
2,9.382295,9.584296
3,10.709936,10.008035
4,9.304687,9.224945
5,8.907966,10.284571
6,8.513493,9.784138
7,10.174744,10.255175
8,11.217429,9.902984


## Relative Mean Absolute Error
> $$ \mathrm{RMAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}^{base}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{base}_{\tau})} $$

![](imgs/losses/rmae_loss.png)

In [ ]:
#| export
def rmae(
    df: DataFrame,
    model_cols1: List[str],
    model_cols2: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DataFrame:
    """Relative Mean Absolute Error (RMAE)
    
    Calculates the RAME between two sets of forecasts (from two different forecasting methods).
    A number smaller than one implies that the forecast in the 
    numerator is better than the forecast in the denominator."""
    numerator = mae(df, model_cols1, id_col, target_col)
    denominator = mae(df, model_cols2, id_col, target_col)
    if isinstance(numerator, pd.DataFrame):
        res = numerator.merge(denominator, on=id_col, suffixes=('', '_denominator'))
        out_cols = [id_col]
        for m1, m2 in zip(model_cols1, model_cols2):
            col_name = f'{m1}_div_{m2}'
            res[col_name] = res[m1] / res[f'{m2}_denominator']
            out_cols.append(col_name)
        res = res[out_cols]
    else:
        res = numerator.join(denominator, on=id_col, suffix='_denominator')
        res = res.select([id_col, *[pl.col(m1).truediv(pl.col(f'{m2}_denominator')).alias(f'{m1}_div_{m2}') for m1, m2 in zip(model_cols1, model_cols2)]])
    return res

In [ ]:
show_doc(rmae)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L315){target="_blank" style="float:right; font-size:smaller"}

### rmae

>      rmae
>            (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFr
>            ame], model_cols1:List[str], model_cols2:List[str],
>            id_col:str='unique_id', target_col:str='y')

Relative Mean Absolute Error (RMAE)

Calculates the RAME between two sets of forecasts (from two different forecasting methods).
A number smaller than one implies that the forecast in the 
numerator is better than the forecast in the denominator.

In [ ]:
rmae(series, models, list(reversed(models)))

,unique_id,model1_div_model2,model2_div_model1
0,0,0.975928,1.024665
1,1,1.087473,0.919563
2,2,0.978924,1.021530
3,3,1.070134,0.934463
4,4,1.008644,0.991430
5,5,0.866149,1.154536
6,6,0.870132,1.149251
7,7,0.992157,1.007905
8,8,1.132732,0.882821
9,9,0.946397,1.056640


In [ ]:
rmae(series_pl, models, list(reversed(models))).sort('unique_id')

unique_id,model1_div_model2,model2_div_model1
i64,f64,f64
0,0.975928,1.024665
1,1.087473,0.919563
2,0.978924,1.02153
3,1.070134,0.934463
4,1.008644,0.99143
5,0.866149,1.154536
6,0.870132,1.149251
7,0.992157,1.007905
8,1.132732,0.882821


# <span style="color:DarkOrange">4. Probabilistic Errors </span>

## Quantile Loss
> $$ \mathrm{QL}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{(q)}_{\tau}) = 
        \frac{1}{H} \sum^{t+H}_{\tau=t+1} 
        \Big( (1-q)\,( \hat{y}^{(q)}_{\tau} - y_{\tau} )_{+} 
        + q\,( y_{\tau} - \hat{y}^{(q)}_{\tau} )_{+} \Big) $$

![](imgs/losses/q_loss.png)

In [ ]:
#| export
@_base_docstring
def quantile_loss(
    df: DataFrame,
    model_cols: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DataFrame:
    """Quantile Loss (QL)
    
    QL measures the deviation of a quantile forecast.
    By weighting the absolute deviation in a non symmetric way, the
    loss pays more attention to under or over estimation.    
    A common value for q is 0.5 for the deviation from the median."""
    _metric_protections(y, y_hat, weights)

    delta_y = y - y_hat
    loss = np.maximum(q * delta_y, (q - 1) * delta_y)

    if weights is not None:
        quantile_loss = np.average(loss[~np.isnan(loss)], 
                             weights=weights[~np.isnan(loss)],
                             axis=axis)
    else:
        quantile_loss = np.nanmean(loss, axis=axis)
        
    return quantile_loss

In [ ]:
show_doc(quantile_loss)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L355){target="_blank" style="float:right; font-size:smaller"}

### quantile_loss

>      quantile_loss
>                     (df:Union[pandas.core.frame.DataFrame,polars.dataframe.fra
>                     me.DataFrame], model_cols:List[str],
>                     id_col:str='unique_id', target_col:str='y')

Quantile Loss (QL)

QL measures the deviation of a quantile forecast.
By weighting the absolute deviation in a non symmetric way, the
loss pays more attention to under or over estimation.    
A common value for q is 0.5 for the deviation from the median.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| model_cols | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **Union** |  | **dataframe with the QUANTILE_LOSS for each id.** |

## Multi-Quantile Loss
> $$ \mathrm{MQL}(\mathbf{y}_{\tau},
                    [\mathbf{\hat{y}}^{(q_{1})}_{\tau}, ... ,\hat{y}^{(q_{n})}_{\tau}]) = 
       \frac{1}{n} \sum_{q_{i}} \mathrm{QL}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{(q_{i})}_{\tau}) $$

![](imgs/losses/mq_loss.png)

In [ ]:
#| export
def mqloss(
    y: np.ndarray,
    y_hat: np.ndarray, 
    quantiles: np.ndarray, 
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """ Multi-Quantile loss (MQL)
    
    MQL calculates the average multi-quantile Loss for
    a given set of quantiles, based on the absolute 
    difference between predicted quantiles and observed values.

    The limit behavior of MQL allows to measure the accuracy 
    of a full predictive distribution $\mathbf{\hat{F}}_{\\tau}$ with 
    the continuous ranked probability score (CRPS). This can be achieved 
    through a numerical integration technique, that discretizes the quantiles 
    and treats the CRPS integral with a left Riemann approximation, averaging over 
    uniformly distanced quantiles.    

    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat : numpy array
        Predicted values.    
    quantiles : numpy array
        Quantiles to compare against.
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the elements of 
        the input array. If axis is negative it counts from the last to first.

    Returns
    -------
    numpy array or double
        MQL along the specified axis.

    References
    ----------
    [1] https://www.jstor.org/stable/2629907
    """ 
    if weights is None: weights = np.ones(y.shape)
        
    _metric_protections(y, y_hat, weights)
    n_q = len(quantiles)
    
    y_rep  = np.expand_dims(y, axis=-1)
    error  = y_rep - y_hat
    mqloss = np.maximum(quantiles * error, (quantiles - 1) * error)
    
    # Match y/weights dimensions and compute weighted average
    weights = np.repeat(np.expand_dims(weights, axis=-1), repeats=n_q, axis=-1)
    mqloss  = np.average(mqloss, weights=weights, axis=axis)

    return mqloss

In [ ]:
show_doc(mqloss)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L404){target="_blank" style="float:right; font-size:smaller"}

### mqloss

>      mqloss (y:numpy.ndarray, y_hat:numpy.ndarray, quantiles:numpy.ndarray,
>              weights:Optional[numpy.ndarray]=None, axis:Optional[int]=None)

Multi-Quantile loss (MQL)

MQL calculates the average multi-quantile Loss for
a given set of quantiles, based on the absolute 
difference between predicted quantiles and observed values.

The limit behavior of MQL allows to measure the accuracy 
of a full predictive distribution $\mathbf{\hat{F}}_{\tau}$ with 
the continuous ranked probability score (CRPS). This can be achieved 
through a numerical integration technique, that discretizes the quantiles 
and treats the CRPS integral with a left Riemann approximation, averaging over 
uniformly distanced quantiles.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat | ndarray |  | Predicted values.     |
| quantiles | ndarray |  | Quantiles to compare against. |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the elements of <br>the input array. If axis is negative it counts from the last to first. |
| **Returns** | **Union** |  | **MQL along the specified axis.** |

## Coverage

In [ ]:
#| export
def coverage(
    y: np.ndarray,
    y_hat_lo: np.ndarray,
    y_hat_hi: np.ndarray, 
) -> Union[float, np.ndarray]:
    """
    Coverage of y with y_hat_lo and y_hat_hi. 
    
    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat_lo : numpy array
        Lower prediction interval.
    y_hat_hi : numpy array
        Higher prediction interval.

    Returns
    -------
    numpy array or double
        Coverage of y_hat

    References
    ----------
    [1] https://www.jstor.org/stable/2629907            
    """ 
    return 100 * np.logical_and(y>=y_hat_lo, y<=y_hat_hi).mean()

In [ ]:
show_doc(coverage)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L465){target="_blank" style="float:right; font-size:smaller"}

### coverage

>      coverage (y:numpy.ndarray, y_hat_lo:numpy.ndarray,
>                y_hat_hi:numpy.ndarray)

Coverage of y with y_hat_lo and y_hat_hi.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| y | ndarray | Observed values. |
| y_hat_lo | ndarray | Lower prediction interval. |
| y_hat_hi | ndarray | Higher prediction interval. |
| **Returns** | **Union** | **Coverage of y_hat** |

## Calibration

In [ ]:
#| export
def calibration(
    y: np.ndarray,
    y_hat_hi: np.ndarray, 
) -> Union[float, np.ndarray]:
    """
    Fraction of y that is lower than y_hat_hi. 
    
    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat_hi : numpy array
        Higher prediction interval.

    Returns
    -------
    numpy array or double
        Calibration of y_hat
        
    References
    ----------
    [1] https://www.jstor.org/stable/2629907            
    """ 
    return (y<=y_hat_hi).mean()

In [ ]:
show_doc(calibration)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L494){target="_blank" style="float:right; font-size:smaller"}

### calibration

>      calibration (y:numpy.ndarray, y_hat_hi:numpy.ndarray)

Fraction of y that is lower than y_hat_hi.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| y | ndarray | Observed values. |
| y_hat_hi | ndarray | Higher prediction interval. |
| **Returns** | **Union** | **Calibration of y_hat** |

## CRPS
> $$ \mathrm{sCRPS}(\hat{F}_{\tau}, \mathbf{y}_{\tau}) = \frac{2}{N} \sum_{i}
    \int^{1}_{0}
    \frac{\mathrm{QL}(\hat{F}_{i,\tau}, y_{i,\tau})_{q}}{\sum_{i} | y_{i,\tau} |} dq $$
>
> Where $\hat{F}_{\tau}$ is the an estimated multivariate distribution, and $y_{i,\tau}$
    are its realizations. 

In [ ]:
#| export
def scaled_crps(
    y: np.ndarray,
    y_hat: np.ndarray, 
    quantiles: np.ndarray, 
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """Scaled Continues Ranked Probability Score
    
    Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
    to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.
    This metric averages percentual weighted absolute deviations as 
    defined by the quantile losses.


    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat : numpy array
        Predicted values.    
    quantiles : numpy array
        Quantiles to compare against.
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the elements of 
        the input array. If axis is negative it counts from the last to first.

    Returns
    -------
    numpy array or double.
        Scaled crps along the specified axis.

    References
    ----------
    [1] https://proceedings.mlr.press/v139/rangapuram21a.html      
    """ 
    eps = np.finfo(float).eps
    norm  = np.sum(np.abs(y))
    loss  = mqloss(y=y, y_hat=y_hat, quantiles=quantiles, weights=weights, axis=axis)
    loss  = 2 * loss * np.sum(np.ones(y.shape)) / (norm + eps)
    return loss

In [ ]:
show_doc(scaled_crps)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L520){target="_blank" style="float:right; font-size:smaller"}

### scaled_crps

>      scaled_crps (y:numpy.ndarray, y_hat:numpy.ndarray,
>                   quantiles:numpy.ndarray,
>                   weights:Optional[numpy.ndarray]=None,
>                   axis:Optional[int]=None)

Scaled Continues Ranked Probability Score

Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.
This metric averages percentual weighted absolute deviations as 
defined by the quantile losses.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat | ndarray |  | Predicted values.     |
| quantiles | ndarray |  | Quantiles to compare against. |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the elements of <br>the input array. If axis is negative it counts from the last to first. |
| **Returns** | **Union** |  | **Scaled crps along the specified axis.** |